<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Практическая работа № 6**  
## **Трансформеры и BERT архитектуры для табличных данных и временных рядов**

---

### **1. Цель и задачи работы**

**Цель работы** — сформировать у обучающегося целостное и критически осмысленное представление о роли архитектур на основе механизма внимания в задачах анализа структурированных данных. Особое внимание уделяется не прямому переносу языковых моделей, а **специализированной адаптации идей BERT и трансформера** к природе числовых последовательностей и табличных признаков. Работа направлена на освоение методологии проектирования, обучения, интерпретации и сравнения современных моделей, построенных после 2020 года, включая как **специализированные трансформеры**, так и **BERT-вдохновлённые подходы с маскированным самообучением**.

**Основные задачи работы:**  
1. Использовать два типа данных:  
   - **Временной ряд**, обработанный в Практической работе № 1 (не менее 720 последовательных наблюдений, с лагами, скользящими окнами, циклическими кодировками);  
   - **Табличный датасет без временной структуры** (например, California Housing, Adult Income).  
2. Реализовать и сравнить следующие категории моделей:  
   - **Специализированные трансформеры для временных рядов**:  
     - Informer (ProbSparse Attention),  
     - Autoformer (автокорреляция),  
     - FEDformer (частотно-доменный attention),  
     - TimesNet (1D→2D периодическое преобразование),  
     - TSMixer (MLP-смеси по времени и признакам);  
   - **BERT-вдохновлённые модели для временных рядов**:  
     - PatchTST (представление ряда как последовательности «патчей» + Transformer encoder),  
     - TimeBERT (маскированное обучение с временными эмбеддингами),  
     - TS2Vec (контрастивное обучение временных представлений);  
   - **Трансформеры для табличных данных**:  
     - TabTransformer (Transformer над эмбеддингами категориальных признаков),  
     - FT-Transformer (унифицированная токенизация всех признаков).  
3. Для каждой модели:  
   - Обеспечить корректную предобработку (масштабирование, кодирование категорий, разбиение на патчи или окна);  
   - Применить **строгое разделение данных**: для временных рядов — `TimeSeriesSplit`, для табличных — стратифицированное разбиение;  
   - Исключить **утечку будущего**, особенно в моделях с маскированием.  
4. Провести обучение в двух режимах:  
   - **Supervised-only** (для моделей без самообучения);  
   - **Self-supervised pretraining + fine-tuning** (для BERT-подобных моделей).  
5. Оценить качество:  
   - Для регрессии: MAE, RMSE, MASE, SMAPE;  
   - Для классификации: Accuracy, F1, AUC-ROC — **только на независимой тестовой выборке**.  
6. Проанализировать:  
   - **Вычислительную эффективность**: время обучения, память, масштабируемость по длине последовательности;  
   - **Интерпретируемость**: attention maps, важность патчей, вклад категориальных признаков;  
   - **Эффективность самообучения**: прирост качества при ограниченных labelled данных.  
7. Сравнить все модели между собой и с **лучшими решениями из Практических работ № 3 (классический ML) и № 5 (глубокое обучение без трансформеров)**.  
8. Разработать модульную кодовую базу на PyTorch с единым интерфейсом для всех архитектур.  
9. Опубликовать воспроизводимый пайплайн: код, конфигурации, веса, визуализации, сравнительные таблицы.

---

### **2. Теоретические предпосылки**

Появление BERT в 2018 году ознаменовало сдвиг от поверхностного к **контекстуальному представлению данных**. Однако его прямое применение к числовым последовательностям методологически некорректно: временные ряды не обладают словарём, грамматикой или семантикой, свойственными естественному языку. Тем не менее, **ключевые идеи BERT** — маскированное моделирование, глубокий encoder, позиционная осведомлённость — могут быть **переосмыслены** для структурированных данных.

Современные архитектуры решают эту задачу по-разному.  
- **PatchTST** трактует временной ряд как последовательность «токенов», где токеном выступает **патч** — непересекающееся окно из k точек. Это позволяет применить стандартный Transformer encoder, но с **улучшенной позиционной кодировкой**, учитывающей как глобальный, так и локальный порядок.  
- **TimeBERT** вводит **богатые временные эмбеддинги**: абсолютное время, относительное смещение, циклические компоненты (день недели, месяц), что даёт модели понимание временной структуры. Обучение ведётся через восстановление маскированных наблюдений — аналог MLM в BERT.  
- **TS2Vec** отказывается от attention в пользу **контрастивного обучения**: близкие по времени сегменты должны иметь схожие представления в скрытом пространстве, что обеспечивает устойчивость к шуму и обобщение.

Для **специализированных трансформеров** характерен отказ от dogmatic self-attention:  
- **Autoformer** заменяет его на **автокорреляцию** — естественный механизм для временных зависимостей;  
- **FEDformer** работает в **частотной области**, выделяя глобальные тренды и сезонности;  
- **TimesNet** преобразует 1D-ряд в 2D-матрицу, чтобы применить **2D-CNN** к обнаруженным периодам;  
- **TSMixer** вообще отказывается от attention, используя **MLP по временным и признаковым измерениям** — простота, скорость, эффективность.

Для **табличных данных** трансформеры применяются через **трансформацию признаков в последовательность**:  
- **TabTransformer** строит attention-взаимодействия только между категориальными признаками, оставляя числовые вне encoder’а;  
- **FT-Transformer** унифицирует все признаки через линейную токенизацию, создавая симметричную архитектуру.

Важно подчеркнуть: **ни одна из этих моделей не использует предобученные веса BERT из Hugging Face**. Все они — **специализированные архитектуры**, обучаемые с нуля на числовых данных. Это принципиальное отличие от наивного «подставить числа вместо слов».

---

### **3. Порядок выполнения работы**

#### **3.1. Подготовка данных**

- **Временной ряд**: формирование окон с учётом горизонта прогноза; для BERT-подобных моделей — разбиение на патчи;  
- **Табличные данные**: числовые признаки → масштабирование; категориальные → эмбеддинги или one-hot;  
- **Маскирование**: для самообучения — случайное скрытие 15–25% патчей или точек (только на обучающей части).

#### **3.2. Реализация моделей**

- Все архитектуры реализуются в едином интерфейсе (`BaseTransformerModel`) с методами `fit()` и `predict()`;  
- Используются официальные или воспроизведённые по статьям реализации;  
- Поддержка GPU и CPU; фиксация random seed для воспроизводимости.

#### **3.3. Обучение и валидация**

- **Временные ряды**:  
  - train = до \( T - 180 \),  
  - val = \( T - 180 : T - 90 \),  
  - test = последние 90 точек (только для финальной оценки);  
- **Табличные данные**: 80/10/10 с стратификацией;  
- **Оптимизация**: AdamW, learning rate scheduling, gradient clipping;  
- **Регуляризация**: dropout, weight decay, early stopping.

#### **3.4. Эксперименты**

- Горизонты прогнозирования: \( h = 1, 7, 30 \);  
- Для BERT-подобных моделей: сравнение с и без pretraining;  
- Фиксация:  
  - Тестовые метрики,  
  - Время обучения (сек),  
  - Потребление памяти (MB),  
  - Стабильность по разным seed.

#### **3.5. Интерпретация**

- **PatchTST/Informer/TabTransformer**: визуализация attention-карт;  
- **TimesNet**: какие периоды активны в CNN-слоях?  
- **TimeBERT**: качество восстановления маскированных точек;  
- **FT-Transformer**: важность признаков через attention weights.

#### **3.6. Сравнение и выводы**

- Сравнительная таблица: «Модель × Тип данных × MAE × Время × Интерпретируемость»;  
- Статистическое сравнение с помощью теста Дикболда–Мариано;  
- Ответ на ключевой вопрос:  
  > **Когда современные трансформеры и BERT-подобные подходы оправдывают свою сложность?**  
- Рекомендации по выбору архитектуры в зависимости от объёма данных, горизонта, требований к интерпретируемости.

#### **3.7. Публикация и воспроизводимость**

- Код — в публичном репозитории (GitHub/GitLab) под лицензией MIT;  
- Данные — под CC BY 4.0;  
- Веса, конфигурации, логи — архивированы;  
- Dockerfile или `requirements.txt` для точного воспроизведения;  
- При наличии — веб-демо на Hugging Face Spaces или Streamlit.

---

### **4. Дополнительные исследовательские задания**

1. **Влияние длины патча**: как качество PatchTST зависит от k = 4, 8, 16, 32?  
2. **Обучение с малым числом меток**: сравнение supervised-only и self-supervised подходов при 10%, 30%, 50% labelled данных.  
3. **Робастность к пропускам**: как модели ведут себя при 10–30% пропущенных значений?  
4. **Гибридизация**: объединение представлений из FT-Transformer и прогнозов LightGBM через стекинг.  
5. **Анализ утечки**: что происходит, если в маскирующей модели случайно разрешить доступ к будущему?

---

### **5. Требования к отчёту**

Отчёт оформляется в соответствии с ГОСТ 7.32–2017 и содержит:  
1. **Введение** — от BERT к специализированным трансформерам; мотивация, связь с предыдущими работами;  
2. **Методология** — описание данных, архитектур, стратегий обучения, предотвращения утечки;  
3. **Результаты** — таблицы, графики прогнозов, attention maps, сравнение с ML/DL;  
4. **Обсуждение** — критический анализ: attention vs. автокорреляция vs. частотный анализ; эффективность самообучения;  
5. **Заключение** — практические рекомендации: когда использовать PatchTST, когда TSMixer, когда не использовать трансформеры вовсе;  
6. **Список литературы** — оригинальные статьи (2020–2024) по каждой модели;  
7. **Приложения** — архитектурные схемы, фрагменты кода, скриншоты визуализаций, сравнительные метрики.

---

### **6. Критерии оценивания**

| Оценка | Критерии |
|--------|----------|
| **Отлично** | Реализованы ≥7 моделей (включая BERT-подобные); корректная валидация без утечки; глубокая интерпретация (attention maps, патчи, эмбеддинги); сравнение с ML/DL; модульный код; полная воспроизводимость; отчёт по ГОСТ. |
| **Хорошо** | Реализованы 4–6 моделей; есть метрики и базовая интерпретация; отсутствуют лишь отдельные элементы (например, сравнение с прошлыми работами). |
| **Удовлетворительно** | Реализованы 2–3 модели (например, только Informer и PatchTST); есть прогноз, но без диагностики утечки или интерпретации. |
| **Неудовлетворительно** | Применён стандартный BERT из Hugging Face без адаптации; нет разделения данных; отчёт не предоставлен. |
